In [92]:
import pandas as pd

In [93]:
# Считываем исходную таблицу ингредиентов из файла.
ingredients = pd.read_csv('./ingr_df.csv')

In [94]:
ingredients.rename(columns = {'ingr_id':'id_ingr', 
'ingr_name':'name_ingr'}, inplace = True )

In [95]:
# Считываем исходную таблицу рецептов из файла и формируем ее под структуру db
connections = pd.read_csv('./recipe_df.csv')
connections.rename(columns = {
        'recipe_id':'id_recipe',
        'recipe_name':'name_recipe',
        'text':'text_recipe',
        'quantity':'ingr_qty',
        'measure':'unit',
        'ingr_qwn':'ingr_qty_unit',
        'proteinContent':'protein',
        'fatContent':'fat',
        'carbohydrateContent':'carbo'
        }, inplace = True )

In [96]:
# Заменяем ingr_name в таблице connections на id_ingr из таблицы ingredients
mapper = dict(zip(ingredients.name_ingr, ingredients.id_ingr))
connections['id_ingr'] = connections.ingr_name.apply(lambda x: mapper[x])
connections.drop(['ingr_name'], axis=1, inplace=True)
# Формируем столбец id_con в таблице connections из индекса, это будет PK всей db
connections['id_con'] = list(connections.index)

In [97]:
def get_unit(un_list):
    """
    Функция формирования названия единиц измерения.
        Args:
            un_list(list): список слов, 
            начиная со второго в описании ингредиента
            
        Returns:
            unit(str): название единицы измерения
            в именительном падеже единственного числа
        """
    # Словарь замены падежей для единиц измерения ингредиентов
    subst={
            'туки' : 'тука',
            'ожек' : 'ожка',
            'ожки' : 'ожка',
            'штук' : 'штука',
            'чика' : 'чик',
            'уска' : 'усок',
            'иков' : 'ик',
            'кана' : 'кан',
            'анки' : 'анка',
            'сков' : 'сок',
            'ебля' : 'ебель',
            'овки' : 'овка',
            'блей' : 'бель',
            'чков' : 'чок',
            'анов' : 'ан',
            'овок' : 'овка',
            'анок' : 'анка',
            'учка' : 'учок',
            'овых' : 'овая',
            'овые' : 'овая',
            'йных' : 'йная',
            'йные' : 'йная',
            'ьших' : 'ьшой/большая'
             }
    ul=[]
    for word in un_list:
        
        if len(word)>3:
            if word[-4:] in subst.keys():
                subst_key = word[-4:]
                word = word[:-4]+subst[subst_key]
            
        ul.append(word)
    unit = (' ').join(ul)
    return unit

In [98]:
# Разбиваем описание ингредиента на список слов
connections['list'] = connections.ingr_qty_unit.str.split(' ')
# Формируем столбец begin из первых слов описания ингредиента
connections['begin'] = connections.list.apply(lambda x: x[0])
# Формируем столбец end из списка остальных слов описания ингредиента
connections['end'] = connections.list.apply(lambda x: x[1:])
# Заменяем символы дробей в столбце begin из первых слов описания ингредиента
connections.begin = connections.begin.str.replace('½','0.5').replace('¼','0.25').replace(
    '⅓','0.33').replace('⅔','0.67').replace('¾','0.75').replace(',','.', regex=True)
# Формируем признак isalpha - первые слов описания ингредиента состоят из букв
connections['isalpha'] = connections.begin.str.isalpha()
# Формируем признак unit - названия единиц измерения
connections['unit'] = connections['end'].apply(get_unit)
# Формируем признак qty - количество ингредиента
connections['ingr_qty'] = connections.apply(lambda row : float(row.begin) if row.isalpha==False else 0, axis = 1)
# Уточняем признак unit: для текстовых названий единиц измерения используем всю исходную строку
connections['unit'] = connections.apply(lambda row: row.ingr_qty_unit if row.ingr_qty==0 else row.unit, axis = 1)
# Удаление временных признаков
connections.drop(['list', 'begin', 'end', 'isalpha'], axis=1, inplace=True)

In [99]:
# Создаем таблицу unit
unit = pd.DataFrame(connections.unit.unique(), columns=['name_unit'])
unit['id_unit'] = list(unit.index)

In [100]:
# Заменяем названия ингредиентов в таблице connections на их id
mapper = dict(zip(unit.name_unit,unit.id_unit))
connections.unit = connections.unit.apply(lambda x: mapper[x])
connections.rename(columns = {'unit':'id_unit'}, inplace = True )

In [101]:
connections.head(1)

,id_recipe,cuisine,meal_mode,name_recipe,portions,calories,protein,fat,carbo,ingr_qty_unit,id_unit,ingr_qty,text_recipe,id_ingr,id_con
0,0,Русская кухня,Завтраки,Сырники из творога,2,938,42,61.0,56.0,350 г,0,350.0,Положите весь творог в кастрюльку и разомните ...,0,0


In [102]:
# Создаем таблицу recipe
recipe = connections.copy(deep=True)
recipe.drop(['ingr_qty_unit', 'id_unit', 'ingr_qty', 'id_con', 'id_ingr'], axis=1, inplace=True)
recipe=recipe.groupby(by='id_recipe').first().reset_index()
recipe['vegeterian'] = False
recipe.rename(columns = {'meal_mode':'meal_type'}, inplace = True )

In [103]:
recipe.head(1)

,id_recipe,cuisine,meal_type,name_recipe,portions,calories,protein,fat,carbo,text_recipe,vegeterian
0,0,Русская кухня,Завтраки,Сырники из творога,2,938,42,61.0,56.0,Положите весь творог в кастрюльку и разомните ...,False


In [104]:
connections.sample(2)

,id_recipe,cuisine,meal_mode,name_recipe,portions,calories,protein,fat,carbo,ingr_qty_unit,id_unit,ingr_qty,text_recipe,id_ingr,id_con
4780,550,Китайская кухня,Основные блюда,Свинина в кисло-сладком соусе,2,1486,36,95.0,124.0,400 г,0,400.0,Первым делом нужно приготовить сам соус. Для э...,642,4780
5953,689,Шведская кухня,Основные блюда,Стейк из тунца с гарниром из брюссельской капусты,2,461,66,16.0,15.0,1 мл,7,1.0,"Сделать маринад, смешав в глубокой миске 10 мл...",13,5953


In [105]:
# После создания таблицы recipes удаляем лишние признаки из таблицы set_ingredients
connections.drop(['name_recipe', 'portions', 'calories', 'protein', 
                  'fat', 'carbo', 'text_recipe', 'cuisine', 'meal_mode'], axis=1, inplace=True)

In [106]:
connections.sample(1)

,id_recipe,ingr_qty_unit,id_unit,ingr_qty,id_ingr,id_con
2645,292,1 штука,1,1.0,1,2645


In [108]:
# Запись в файлы
connections.to_csv('./connections.csv', index=False)
recipe.to_csv('./recipe.csv', index=False)
unit.to_csv('./unit.csv', index=False)
ingredients.to_csv('./ingredient.csv', index=False)

### Визуализация таблиц

In [111]:
connections.sample(5)

,id_recipe,ingr_qty_unit,id_unit,ingr_qty,id_ingr,id_con
4437,505,1 штука,1,1.0,617,4437
5461,630,5 столовых ложек,2,5.0,2,5461
100,10,50 г,0,50.0,55,100
5161,597,1 чайная ложка,4,1.0,186,5161
4433,504,по вкусу,6,0.0,22,4433


In [113]:
recipe.sample(5)

,id_recipe,cuisine,meal_type,name_recipe,portions,calories,protein,fat,carbo,text_recipe,vegeterian
582,587,Итальянская кухня,Закуски,Креветки в соусе песто,4,901,59,67.0,11.0,"В блендере смешать петрушку, базилик, оливково...",False
695,705,Итальянская кухня,Основные блюда,Сальтимбокка,4,460,41,28.0,1.0,Разогреть духовку до 250 градусов. Отбить куск...,False
604,609,Итальянская кухня,Выпечка и десерты,Маффины с бананом и сгущенкой,8,224,4,6.0,40.0,"Смешать муку, сахар, соль, пряность. Добавить ...",False
643,648,Русская кухня,Основные блюда,Картофельные котлеты на растительном масле,4,419,10,20.0,52.0,Картофель нужно очистить и сварить до мягкости...,False
321,321,Русская кухня,Основные блюда,Котлеты из индейки с соусом беарнез,10,986,43,83.0,16.0,Индейку пропустить через мясорубку два раза вм...,False


In [114]:
unit

,name_unit,id_unit
0,г,0
1,штука,1
2,столовая ложка,2
3,стакан,3
4,чайная ложка,4
5,зубчик,5
6,по вкусу,6
7,мл,7
8,головка,8
9,кусок,9


In [115]:
ingredients

,id_ingr,name_ingr
0,0,Творог
1,1,Куриное яйцо
2,2,Пшеничная мука
3,3,Сахар
4,4,Подсолнечное масло
...,...,...
754,754,Чабер
755,755,Паста каннеллони
756,756,Мороженое
757,757,Сливочный ликер
